In [26]:
import numpy as np
import pandas as pd
from scipy.linalg import solve

In [27]:
def QRdec(A):
    (r, c) = np.shape(A)
    Q = np.identity(r)
    R = np.copy(A)
    (rows, cols) = np.tril_indices(r, -1, c)
    for (row, col) in zip(rows, cols):
        if R[row, col] != 0:
            temp = np.sqrt(R[col, col]**2+R[row, col]**2)  
            cos = R[col, col]/temp
            sin = -R[row, col]/temp
            T = np.identity(r)
            T[[col, row], [col,row]] = cos
            T[row, col] = sin
            T[col, row] = -sin
            R = T.dot(R)
            Q = Q.dot(T.T)
    return (Q, R)

In [43]:
#LU test good
a = np.array([[4,2,0,0],[2,5,2,0],[0,2,6,2],[0,0,2,7]])
b=np.array([-0.393, -0.389,-0.393, -0.389])
a
Q=QRdec(a)[0]
R=QRdec(a)[1]
print(Q)
print(R)
#Q = np.dot(Q,Q.T)
#print(Q)

[[ 0.89442719 -0.39036003  0.1983799  -0.09090909]
 [ 0.4472136   0.78072006 -0.3967598   0.18181818]
 [ 0.          0.48795004  0.7935196  -0.36363636]
 [ 0.          0.          0.41659779  0.90909091]]
[[ 4.47213595e+00  4.02492236e+00  8.94427191e-01  0.00000000e+00]
 [ 0.00000000e+00  4.09878031e+00  4.48914034e+00  9.75900073e-01]
 [ 0.00000000e+00 -2.01858732e-16  4.80079359e+00  4.50322373e+00]
 [ 0.00000000e+00  9.25032918e-17  0.00000000e+00  5.63636364e+00]]


In [29]:
#returns array with cond number
def Ncond(A):
#spectral
    normA=np.linalg.norm(A, ord=1)
    A_1=np.linalg.inv(A)
    normA_1=np.linalg.norm(A_1, ord=1)
    conds=normA*normA_1
#ortega
    number=len(A)
    condv=1
    for i in range(number):
        colsum=0
        for j in range(number):
            colsum=colsum+A[i][j]
        colsum=np.sqrt(abs(colsum))
        condv=condv*colsum
    detA=abs(np.linalg.det(A))
    condv=condv/detA
#angle
    pros=[]
    for i in range(number):
        Asum=0
        A_1sum=0
        for j in range(number):
            Asum=Asum+A[i][j]*A[i][j]
            A_1sum=Asum+A_1[i][j]*A_1[i][j]
        pros.append(np.sqrt(Asum*A_1sum))
    conda=max(pros)
    
    cond=[]
    cond.append(conds)
    cond.append(condv)
    cond.append(conda)
    return cond

In [30]:
#cond numbers for A,Q,R
def Filling(A):
    Q=QRdec(A)[0]
    R=QRdec(A)[1]
    resA=Ncond(A)
    resQ=Ncond(Q)
    resR=Ncond(R)
    conds=['spectral','volume','angular']
    data = {'name':[],'cond': [], 'num': []}
    df = pd.DataFrame(data)
    
    for j in range(3):
        new_row = pd.DataFrame({'name':'A','cond':conds[j], 'num': resA[j]}, index=[0])
        df = pd.concat([df, new_row], ignore_index= True)
    for j in range(3):
        new_row = pd.DataFrame({'name':'Q','cond':conds[j], 'num': resQ[j]}, index=[0])
        df = pd.concat([df, new_row], ignore_index= True)
    for j in range(3):
        new_row = pd.DataFrame({'name':'R','cond':conds[j], 'num': resR[j]}, index=[0])
        df = pd.concat([df, new_row], ignore_index= True)
    
    print(df)

In [31]:
#LU test good
a = np.array([[4,2,0,0],[2,5,2,0],[0,2,6,2],[0,0,2,7]])
b=np.array([-0.393, -0.389,-0.393, -0.389])
a

array([[4, 2, 0, 0],
       [2, 5, 2, 0],
       [0, 2, 6, 2],
       [0, 0, 2, 7]])

In [32]:
Q=QRdec(a)[0]
R=QRdec(a)[1]
print(Q)
print(R)

[[ 0.89442719 -0.39036003  0.1983799  -0.09090909]
 [ 0.4472136   0.78072006 -0.3967598   0.18181818]
 [ 0.          0.48795004  0.7935196  -0.36363636]
 [ 0.          0.          0.41659779  0.90909091]]
[[ 4.47213595e+00  4.02492236e+00  8.94427191e-01  0.00000000e+00]
 [ 0.00000000e+00  4.09878031e+00  4.48914034e+00  9.75900073e-01]
 [ 0.00000000e+00 -2.01858732e-16  4.80079359e+00  4.50322373e+00]
 [ 0.00000000e+00  9.25032918e-17  0.00000000e+00  5.63636364e+00]]


In [33]:
Filling(a)

  name      cond        num
0    A  spectral   6.048387
1    A    volume   0.140552
2    A   angular  53.013006
3    Q  spectral   3.261218
4    Q    volume   0.868195
5    Q   angular   1.351461
6    R  spectral   6.670395
7    R    volume   0.138368
8    R   angular  43.340489


In [34]:
x0=solve(a,b)
z=solve(Q.dot(R),b)
print(x0)
print(z)

[-0.08466532 -0.02716935 -0.04191129 -0.04359677]
[-0.08466532 -0.02716935 -0.04191129 -0.04359677]


In [35]:
#QR test not so good
a = np.array([[1,0.99],[0.99,0.98]])
b=np.array([-0.393, -0.389])
a

array([[1.  , 0.99],
       [0.99, 0.98]])

In [36]:
Q=QRdec(a)[0]
R=QRdec(a)[1]
print(Q)
print(R)

[[ 0.71065111 -0.7035446 ]
 [ 0.7035446   0.71065111]]
[[ 1.40716026e+00  1.39301830e+00]
 [-5.20162676e-18 -7.10651109e-05]]


In [37]:
Filling(a)

  name      cond           num
0    A  spectral  39601.000000
1    A    volume  19799.747473
2    A   angular  13930.886724
3    Q  spectral      1.999949
4    Q    volume      0.100250
5    Q   angular      1.226795
6    R  spectral  39403.000000
7    R    volume    141.065588
8    R   angular  27582.466213


In [38]:
x0=solve(a,b)
z=solve(Q.dot(R),b)
print(x0)
print(z)

[ 0.3 -0.7]
[ 0.3 -0.7]


In [39]:
#QR test bad
a = np.array([[1.2969, 0.8648],[0.2161,0.1441]])
b=np.array([-0.393, -0.389])
a

array([[1.2969, 0.8648],
       [0.2161, 0.1441]])

In [40]:
Q=QRdec(a)[0]
R=QRdec(a)[1]
print(Q)
print(R)

[[ 0.98640009 -0.16436199]
 [ 0.16436199  0.98640009]]
[[ 1.31478090e+00  8.76723360e-01]
 [-2.27875153e-17  7.60582996e-09]]


In [41]:
Filling(a)

  name      cond           num
0    A  spectral  3.270652e+08
1    A    volume  8.824083e+07
2    A   angular  1.348042e+08
3    Q  spectral  1.324253e+00
4    Q    volume  9.726100e-01
5    Q   angular  1.404630e+00
6    R  spectral  2.881348e+08
7    R    volume  1.291054e+04
8    R   angular  1.385471e+08


In [42]:
x0=solve(a,b)
z=solve(Q.dot(R),b)
print(x0)
print(z)

[ 27977590.04349676 -41956680.06523005]
[ 27977590.04349676 -41956680.06523005]
